# **MIP3_Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """You are tasked with scheduling the power output of 6 electric power thermal units over the timespan of 15 periods. There is a constant startup cost of for each power unit that is applied if the power plant is turned on. The startup costs are 10324€, 5678€, 7802€, 12899€, 4596€ and 9076€ for powerplants 1 to 6, respectively. In addition, there is a constant shutdown cost for each power unit that is applied if the power plant is turned off. The shutdown costs are 2673€, 5893€, 982€, 6783€, 2596€ and 3561€ for powerplants 1 to 6, respectively. There is also a fixed and variable cost applied if the power plant is running. The fixed cost is constant and the variable cost is proportional to the output of a power plant. There are lower and upper bounds for the output power for each unit. The variable costs are different for each power plant and at each time step. Next, there are maximum power increments and decrements for each power plant that limit how much the output can change from one time period to the next. There is a total power demand that needs to be fulfilled by the power units. Finally, for security reasons, the total available power out should always be 10% higher than the demand."""


In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [6]:
print(response1.choices[0].message.content)

1. Decision Variables:
    - X_it: binary variable. 1 if plant i is operating at time t, 0 otherwise, i ∈ {1,2,3,4,5,6}, t ∈ {1,...,15}
    - Y_it: binary variable. 1 if plant i is turned on at time t, 0 otherwise, i ∈ {1,2,3,4,5,6}, t ∈ {1,...,15}
    - Z_it: binary variable. 1 if plant i is shut down at time t, 0 otherwise, i ∈ {1,2,3,4,5,6}, t ∈ {1,...,15}
    - P_it: continuous variable. Power output of plant i at time t, i ∈ {1,2,3,4,5,6}, t ∈ {1,...,15}

2. Parameters:
    - S_i: startup cost of power plant i, i ∈ {1,2,3,4,5,6}
    - D_i: shutdown cost of power plant i, i ∈ {1,2,3,4,5,6}
    - F_i: fixed cost of power plant i, i ∈ {1,2,3,4,5,6}
    - V_it: Variable cost of power plant i at time t, i ∈ {1,2,3,4,5,6}, t ∈ {1,...,15}
    - LB_it, UB_it: Lower and upper bounds of power output for plant i at time t, i ∈ {1,2,3,4,5,6}, t ∈ {1,...,15}
    - Inc_i, Dec_i: Maximum power increment and decrement for plant i, i ∈ {1,2,3,4,5,6}
    - Demand_t: Total power demand at time t, t 

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [8]:
print(response2.choices[0].message.content)

The objective function is to minimize the total cost, which includes the startup costs, the shutdown costs, and the combined fixed and variable costs over the 15 periods of operation for all of the 6 power plants. 

Objective Function Minimize: 
∑_i∑_t (S_i*Y_it + D_i*Z_it + F_i*X_it + V_it*P_it) for all i ∈ {1,2,3,4,5,6} and t ∈ {1,...,15}


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [10]:
print(response3.choices[0].message.content)

1. Power output constraint: The total power output from all units must meet the total power demand at each time period with a security margin of 10%.

    Sum(P_it) for all i ∈ {1,...,6} ≥ 1.1* Demand_t , t ∈ {1,...,15}

2. Power Plant Operation constraint: If the power plant is operating then it must not be shutting down or starting up.

     X_it ≤ 1 - Y_it - Z_it , i ∈ {1,...,6}, t ∈ {1,...,15}

3. Power output bounds constraint: The power output for each plant i must be within its specific upper and lower bounds.

    LB_it <= P_it <= UB_it, i ∈ {1,2,3,4,5,6}, t ∈ {1,...,15}

4. Startup and shutdown constraint: A plant can only be started or shut down if it was or wasn't running in the previous period respectively.

    Y_it ≤ 1 - X_i(t-1) , i ∈ {1,...,6}, t ∈ {2,...,15}
    Z_it ≤ X_i(t-1), i ∈ {1,...,6}, t ∈ {2,...,15}

5. Power increments and decrements constraint: The change in power output from one time period to the next cannot exceed the maximum power increments and decremen

### **Generate Pyomo Code**

In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [12]:
Markdown(response4.choices[0].message.content)

Sure. Below are the python pyomo code and a mock data for the above problem. 

```python
from pyomo.environ import *

# Mock Data - replace it with real values
N_PLANTS = ['Plant_1', 'Plant_2', 'Plant_3', 'Plant_4', 'Plant_5', 'Plant_6']
N_PERIODS = range(1, 16)

# specify cost data and bounds in dictionaries
S = D = F = {p: 10 for p in N_PLANTS} 
V = {(i, t): 10*t for i in N_PLANTS for t in N_PERIODS} 
Demand = {(t): 100*t for t in N_PERIODS}
LB = UB = {(i, t): (20, 200) for i in N_PLANTS for t in N_PERIODS}
Inc = Dec = {p: 10 for p in N_PLANTS}

# Steps to Create Model
model = ConcreteModel()

# Define Sets
model.PLANTS = Set(initialize=N_PLANTS)
model.PERIODS = Set(initialize=N_PERIODS)

# Parameters (Mock Data)
model.S = Param(model.PLANTS, initialize=S)
model.D = Param(model.PLANTS, initialize=D)
model.F = Param(model.PLANTS, initialize=F)
model.V = Param(model.PLANTS, model.PERIODS, initialize=V)
model.Demand = Param(model.PERIODS, initialize=Demand)
model.LB = Param(model.PLANTS, model.PERIODS, initialize=lambda model, p, t: LB[p, t][0])
model.UB = Param(model.PLANTS, model.PERIODS, initialize=lambda model, p, t: UB[p, t][1])
model.Inc = Param(model.PLANTS, initialize=Inc)
model.Dec = Param(model.PLANTS, initialize=Dec)

# Define Variables
model.X = Var(model.PLANTS, model.PERIODS, within=Binary)
model.Y = Var(model.PLANTS, model.PERIODS, within=Binary)
model.Z = Var(model.PLANTS, model.PERIODS, within=Binary)
model.P = Var(model.PLANTS, model.PERIODS, within=NonNegativeReals, bounds=lambda model, i, t: (model.LB[i, t], model.UB[i, t]))

# Objective Function
model.Obj = Objective(expr=sum(model.S[i]*model.Y[i, t] + model.D[i]*model.Z[i, t] + model.F[i]*model.X[i, t] + model.P[i, t]*model.V[i, t] for i in model.PLANTS for t in model.PERIODS))

# Define Constraints
def power_output_constraint(model, t):
    return sum(model.P[i, t] for i in model.PLANTS) >= 1.1 * model.Demand[t]
model.compute_power_output_constraint = Constraint(model.PERIODS, rule=power_output_constraint)

def power_plant_operation_constraint(model, i, t):
    return model.X[i, t] <= 1 - model.Y[i, t] - model.Z[i, t]
model.compute_power_plant_operation_constraint = Constraint(model.PLANTS, model.PERIODS, rule=power_plant_operation_constraint)

def startup_shutdown_constraint(model, i, t):
    if t == model.PERIODS.first():
        return Constraint.Skip
    return model.Y[i, t] <= 1 - model.X[i, t - 1], model.Z[i, t] <= model.X[i, t - 1]
model.compute_startup_shutdown_constraint = Constraint(model.PLANTS, model.PERIODS, rule=startup_shutdown_constraint)

def power_increment_decrement_constraint(model, i, t):
    if t == model.PERIODS.first():
        return Constraint.Skip
    return model.P[i, t] - model.P[i, t - 1] <= model.Inc[i], model.P[i, t - 1] - model.P[i, t] <= model.Dec[i]
model.compute_power_increment_decrement_constraint = Constraint(model.PLANTS, model.PERIODS, rule=power_increment_decrement_constraint)
```

This example assumes there is an external solver available to solve the instance, e.g. CPLEX, GLPK etc. The solution can be obtained by passing the model instance to the solver's `solve()` method. 
Also, please note that the power increments and decrements have been modelled as soft constraints in this code block. If the power increments and decrements must not be violated under any circumstances, then you should model them as hard constraints.


### **Run the code Generated by GPT4**

In [46]:
from pyomo.environ import *

# Mock Data - replace it with real values
N_PLANTS = ['Plant_1', 'Plant_2', 'Plant_3', 'Plant_4', 'Plant_5', 'Plant_6']
N_PERIODS = range(1, 16)

# specify cost data and bounds in dictionaries
S = D = F = {p: 10 for p in N_PLANTS}
V = {(i, t): 10*t for i in N_PLANTS for t in N_PERIODS}
Demand = {(t): 100*t for t in N_PERIODS}
LB = UB = {(i, t): (20, 200) for i in N_PLANTS for t in N_PERIODS}
Inc = Dec = {p: 10 for p in N_PLANTS}

# Steps to Create Model
model = ConcreteModel()

# Define Sets
model.PLANTS = Set(initialize=N_PLANTS)
model.PERIODS = Set(initialize=N_PERIODS)

# Parameters (Mock Data)
model.S = Param(model.PLANTS, initialize=S)
model.D = Param(model.PLANTS, initialize=D)
model.F = Param(model.PLANTS, initialize=F)
model.V = Param(model.PLANTS, model.PERIODS, initialize=V)
model.Demand = Param(model.PERIODS, initialize=Demand)
model.LB = Param(model.PLANTS, model.PERIODS, initialize=lambda model, p, t: LB[p, t][0])
model.UB = Param(model.PLANTS, model.PERIODS, initialize=lambda model, p, t: UB[p, t][1])
model.Inc = Param(model.PLANTS, initialize=Inc)
model.Dec = Param(model.PLANTS, initialize=Dec)

# Define Variables
model.X = Var(model.PLANTS, model.PERIODS, within=Binary)
model.Y = Var(model.PLANTS, model.PERIODS, within=Binary)
model.Z = Var(model.PLANTS, model.PERIODS, within=Binary)
model.P = Var(model.PLANTS, model.PERIODS, within=NonNegativeReals, bounds=lambda model, i, t: (model.LB[i, t], model.UB[i, t]))

# Objective Function
model.Obj = Objective(expr=sum(model.S[i]*model.Y[i, t] + model.D[i]*model.Z[i, t] + model.F[i]*model.X[i, t] + model.P[i, t]*model.V[i, t] for i in model.PLANTS for t in model.PERIODS))

# Define Constraints
def power_output_constraint(model, t):
    return sum(model.P[i, t] for i in model.PLANTS) >= 1.1 * model.Demand[t]
model.compute_power_output_constraint = Constraint(model.PERIODS, rule=power_output_constraint)

def power_plant_operation_constraint(model, i, t):
    return model.X[i, t] <= 1 - model.Y[i, t] - model.Z[i, t]
model.compute_power_plant_operation_constraint = Constraint(model.PLANTS, model.PERIODS, rule=power_plant_operation_constraint)

def startup_shutdown_constraint(model, i, t):
    if t == model.PERIODS.first():
        return Constraint.Skip
    return model.Y[i, t] <= 1 - model.X[i, t - 1], model.Z[i, t] <= model.X[i, t - 1]
model.compute_startup_shutdown_constraint = Constraint(model.PLANTS, model.PERIODS, rule=startup_shutdown_constraint)

def power_increment_decrement_constraint(model, i, t):
    if t == model.PERIODS.first():
        return Constraint.Skip
    return model.P[i, t] - model.P[i, t - 1] <= model.Inc[i], model.P[i, t - 1] - model.P[i, t] <= model.Dec[i]
model.compute_power_increment_decrement_constraint = Constraint(model.PLANTS, model.PERIODS, rule=power_increment_decrement_constraint)

ERROR:pyomo.core:Rule failed when generating expression for Constraint compute_startup_shutdown_constraint with index ('Plant_1', 2):
ValueError: Constraint 'compute_startup_shutdown_constraint[Plant_1,2]' does not have a proper value. Constraint expressions expressed as tuples must contain native numeric types or Pyomo NumericValue objects. Tuple (<pyomo.core.expr.relational_expr.InequalityExpression object at 0x7b20667016c0>, <pyomo.core.expr.relational_expr.InequalityExpression object at 0x7b2066701690>) contained invalid type, InequalityExpression
ERROR:pyomo.core:Constructing component 'compute_startup_shutdown_constraint' from data=None failed:
    ValueError: Constraint 'compute_startup_shutdown_constraint[Plant_1,2]' does not have a proper value. Constraint expressions expressed as tuples must contain native numeric types or Pyomo NumericValue objects. Tuple (<pyomo.core.expr.relational_expr.InequalityExpression object at 0x7b20667016c0>, <pyomo.core.expr.relational_expr.Inequa

ValueError: Constraint 'compute_startup_shutdown_constraint[Plant_1,2]' does not have a proper value. Constraint expressions expressed as tuples must contain native numeric types or Pyomo NumericValue objects. Tuple (<pyomo.core.expr.relational_expr.InequalityExpression object at 0x7b20667016c0>, <pyomo.core.expr.relational_expr.InequalityExpression object at 0x7b2066701690>) contained invalid type, InequalityExpression

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [45]:
from pyomo.environ import *
#data inputted by human


N = 6
T = 15

# specify cost data and bounds in dictionaries
S = [10324, 5678, 7802, 12899, 4596, 9076]
D = [2673, 5893, 982, 6783, 2596, 3561]
F = [2000, 3000, 2500, 4000, 3500, 4500]
V = [[20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35],
      [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
      [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32],
      [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
      [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36],
      [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]
      ]

Demand = [1000, 1200, 1300, 1100, 1500, 1400, 1600, 1300, 1700, 1800, 1900, 1600, 2000, 1800, 1700]
LB = [50, 40, 30, 60, 55, 65]
UB = [500, 600, 550, 700, 650, 750]
Inc = [100, 120, 110, 130, 125, 140]
Dec = [90, 110, 100, 120, 115, 130]
#end
# Steps to Create Model
model = ConcreteModel()

model.N = RangeSet(1, N)
model.T = RangeSet(1, T)

# Define Sets
model.PLANTS = Set(initialize=model.N)
model.PERIODS = Set(initialize=model.T)

# Define Variables
model.X = Var(model.PLANTS, model.PERIODS, within=Binary)
model.Y = Var(model.PLANTS, model.PERIODS, within=Binary)
model.Z = Var(model.PLANTS, model.PERIODS, within=Binary)
model.P = Var(model.PLANTS, model.PERIODS, within=NonNegativeReals, bounds=lambda model, i, t: (LB[i-1], UB[i-1]))

# Objective Function
model.Obj = Objective(expr=sum(S[i-1]*model.Y[i, t] + D[i-1]*model.Z[i, t] + F[i-1]*model.X[i, t] + model.P[i, t]*V[i-1][t-1] for i in model.PLANTS for t in model.PERIODS))

# Define Constraints
def power_output_constraint(model, t):
    return sum(model.P[i, t] for i in model.PLANTS) >= 1.1 * Demand[t-1]
model.compute_power_output_constraint = Constraint(model.PERIODS, rule=power_output_constraint)

def power_plant_operation_constraint(model, i, t):
    return model.X[i, t] <= 1 - model.Y[i, t] - model.Z[i, t]
model.compute_power_plant_operation_constraint = Constraint(model.PLANTS, model.PERIODS, rule=power_plant_operation_constraint)

def startup_shutdown_constraint(model, i, t):
    if t == model.PERIODS.first():
        return Constraint.Skip
    return model.Y[i, t] <= 1 - model.X[i, t - 1]
model.compute_startup_shutdown_constraint = Constraint(model.PLANTS, model.PERIODS, rule=startup_shutdown_constraint)

def startup_shutdown_constraint1(model, i, t):
    if t == model.PERIODS.first():
        return Constraint.Skip
    return model.Z[i, t] <= model.X[i, t - 1]
model.compute_startup_shutdown_constraint1 = Constraint(model.PLANTS, model.PERIODS, rule=startup_shutdown_constraint1)

def power_increment_decrement_constraint(model, i, t):
    if t == model.PERIODS.first():
        return Constraint.Skip
    return model.P[i, t] - model.P[i, t - 1] <= Inc[i-1]
model.compute_power_increment_decrement_constraint = Constraint(model.PLANTS, model.PERIODS, rule=power_increment_decrement_constraint)

def power_increment_decrement_constraint1(model, i, t):
    if t == model.PERIODS.first():
        return Constraint.Skip
    return model.P[i, t - 1] - model.P[i, t] <= Dec[i-1]
model.compute_power_increment_decrement_constraint1 = Constraint(model.PLANTS, model.PERIODS, rule=power_increment_decrement_constraint1)

SolverFactory('glpk').solve(model)
print(model.Obj())

662805.0
